# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() #class열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [33]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) #dt=decision tree
print(scores)

{'fit_time': array([0.        , 0.        , 0.        , 0.00199294, 0.00199413]), 'score_time': array([0.        , 0.        , 0.        , 0.00099802, 0.0009973 ]), 'test_score': array([0.8375    , 0.85384615, 0.8719923 , 0.83926853, 0.8267565 ])}


In [34]:
import numpy as np

print(np.mean(scores['test_score']))

0.8458726956392981


In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) #n_splits=10: n_splits매개변수는 몇(k) 폴드 교차검증을 할지 정함
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

#GridSearchCV: 하이퍼파라미터(=사용자가 지정해야만 하는 파라미터) 탐색과 교차검증을 한 번에 수행. GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦
#GridSearchCV 사용 시 매개변수를 일일이 바꿔가며 교차검증을 수행하지 않고, 매개변수 값을 나열하면 자동으로 교차검증을 수행해서 최상의 매개변수를 찾을 수 있음


In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #for문 대신 파라미터. -1로 지정 시 시스템에 있는 모든 코어를 사용 (기본값 1)

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_ #그리드 서치로 찾은 최적의 매개변수는 best_params_속성에 저장되어 있음
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) #argmax(): 가장 큰 값의 인덱스를 추출 -> 인덱스를 사용해 params키에 저장된 매개변수를 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [35]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), #노드를 분할하기 위한 불순도 감소 최소량 지정. 0.0001에서 시작하여 0.001이 될 때까지 0.0001을 계속 더한 배열
          'max_depth': range(5, 20, 1), #5에서 20까지 1씩 증가하면서 15개의 값을 만듦
          'min_samples_split': range(2, 100, 10) #샘플 최소 2개. 2에서 100까지 10씩 증가하면서 10개의 값을 만듦
          }

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
print(gs.best_params_) #최상의 교차 검증 점수 확인

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [22]:
from scipy.stats import uniform, randint #uniform은 실숫값을, randint는 정숫값을 뽑음

#랜덤 서치(Random Search)에는 매개변수 값의 목록이 아닌 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달
#연속된 매개변수 값을 탐색할 때 유용. 지정된 횟수만큼 샘플링하요 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량 조절 가능
#교차 검증: 훈련한 모델의 성능을 안정적으로 평가하기 위해 검증세트(=개발세트)를 한 번 나누어 모델을 평가하는 것에 그치지 않고 여러 번 반복

In [23]:
rgen = randint(0, 10) #0~10 사이 10개의 숫자를 샘플링
rgen.rvs(10)

array([6, 9, 1, 5, 7, 6, 5, 1, 7, 7], dtype=int64)

In [24]:
np.unique(rgen.rvs(1000), return_counts=True) #10 -> 1000개로 어느 정도 고르게 추출된 걸 확인

#return_train_score=True로 지정 시 훈련 세트의 점수도 반환(기본값은 False)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([103,  94, 110,  94,  96,  83, 102, 104, 114, 100], dtype=int64))

In [25]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.68643044, 0.36889606, 0.26091513, 0.89176771, 0.66607164,
       0.93869259, 0.14537848, 0.47037062, 0.08286622, 0.72648612])

In [26]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), #0.0001~0.001 사이의 실숫값을 샘플링. 샘플링 횟수는 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [27]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42) #100번(n_iter 매개변수) 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾음
gs.fit(train_input, train_target)

#n_jobs는 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정(기본값은 1). -1로 지정 시 시스템에 있는 모든 코어를 사용
#RandomizedSearchCV: 교차 검증으로 랜덤한 하이퍼파라미터 탐색 => 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B8059F2820>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002B811EB4550>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B811F7EB50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B811ECC0D0>},
                   random_state=42)

In [28]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [30]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target)) #수동으로 매개변수를 바꾸는 대신에, 그리드서치나 랜덤서치를 사용

0.86


## 확인문제

In [31]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B8059F2820>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002B811EB4550>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B811F7EB50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B811ECC0D0>},
                   random_state=42)

In [32]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
